In [ ]:
import numpy as np
import pandas as pd

In [ ]:
books = pd.read_csv("clean_book.csv")

In [ ]:
books.columns

In [ ]:
books.categories.value_counts().shape

In [ ]:
books['categories'].value_counts().reset_index().query("count>=50")

In [ ]:
category_mapping ={
    'Fiction':"Fiction",
    "Juvenile Fiction": "Children's Fiction",
    "Biography & Autobiography": "NonFiction",
    "History":"NonFiction",
    "Literary Criticism":"NonFiction",
    "Philosophy":"NonFiction",
    "Religion":"NonFiction",
    "Comics & Graphic Novels":"Fiction",
    "Drama":"Fiction",
    "Science":"NonFiction",
    "Juvenile Nonfiction":"Children's NonFiction",
    "Poetry":"Fiction"
}

In [ ]:
books['modified_category'] = books['categories'].map(category_mapping)

In [ ]:
books[~books['modified_category'].isna()].shape

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel,Field
from enum import Enum

In [ ]:
class CategoryType(str, Enum):
    FICTION = "Fiction"
    NONFICTION = "NonFiction"

class Blueprint(BaseModel):
    category: CategoryType = Field(description="Category of the Provided Description")

In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
parser = PydanticOutputParser(pydantic_object=Blueprint)

In [ ]:
prompt = PromptTemplate(
    template="Provided the Description of a book Categorize it as either Fiction or NonFiction. Description : \n {description} \n {format_instructions}",
    input_variables=['description'],
    validate_template=True,
    partial_variables= {'format_instructions':parser.get_format_instructions()}
)

In [ ]:
chain = prompt | model |parser

In [ ]:
example1 = books.loc[books["modified_category"] == "Fiction","description"].reset_index(drop=True)[0]
ex2=books.loc[books["modified_category"] == "NonFiction","description"].reset_index(drop=True)[0]

In [ ]:
result=chain.invoke({'description':ex2})

In [ ]:
result.category.value

In [ ]:
def generate_prediction(task):
    chain = prompt | model |parser
    result =chain.invoke({'description':task})
    return result.category.value

In [ ]:
generate_prediction(ex2)

In [ ]:
from tqdm import tqdm

In [ ]:
isbn=[]
predicted_cats=[]

missing_categ = books.loc[books['modified_category'].isna(),['isbn13','description']].reset_index(drop=True)

In [ ]:
books['categories'].isna().sum()

In [ ]:
missing_categ['isbn13']

In [ ]:
for i in tqdm(range(0,len(missing_categ))):
    seq= missing_categ['description']
    predicted_cats += [generate_prediction(seq)]
    isbn+= [missing_categ['isbn13'][i]]

In [ ]:
df=pd.DataFrame({"isbn":isbn,"category":predicted_cats})

In [ ]:
df.shape

In [ ]:
df.duplicated().sum()

In [ ]:
df['category'].value_counts()

In [ ]:
len(isbn)

In [ ]:
len(predicted_cats)

In [ ]:
new =pd.merge(missing_categ,df,left_on="isbn13",right_on="isbn",how="left")

In [ ]:
new[new['isbn'].isna()]

In [ ]:
new[new['isbn'] != new['isbn13']].shape

In [ ]:
new_book= pd.merge(books,new,on="isbn13",how="left")

In [ ]:
new_book.sample(4)

In [ ]:
new_book['modified_category'] = np.where(
    new_book['modified_category'].isna() , new_book['category'],new_book['modified_category']
)

In [ ]:
new_book.isna().sum()

In [ ]:
new_book.drop(['isbn','category','description_y',],inplace=True,axis=1)

In [ ]:
new_book.to_csv("new_book.csv",index=False)